# INFO - EstNLTK NERTagger mudelid sündmuste tuvastamiseks
EstNLTK 1.6 arendusharus oleva [NERTaggeri](https://github.com/estnltk/estnltk/tree/devel_1.6/estnltk/taggers/estner) ümbertreenimine sündmuste tuvastamisele ja mudelite loomine.  
Tulemuste juures on välja toodud 10 alamhulga ristvalideerimisel saadud tulemuste keskmised ja [Nervaluate](https://pypi.org/project/nervaluate/) analüüsi tulemused.

**NB!** Antud koodi jooksutati lokaalses arvutis. Kui on soovi seda Colab'is kasutada, siis tuleb arendusharu NERTagger üles laadida Drive'i ja Colab'i keskkonnas ära asendada (näide sellest on BERT embeddingutega ja ajaseoseid tuvastavate mudelite loomisega tegelevas failis).

# Imports

In [ ]:
from estnltk import Text
from estnltk.converters import text_to_json, json_to_text
import json
import os
from estnltk.taggers import NerTagger, WordLevelNerTagger
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, accuracy_score, precision_score, recall_score
import numpy as np
from sklearn.model_selection import KFold

# Failide lugemine

In [ ]:
def text_from_json(filename):
  data = None
  with open(drive_path + "artikklid/" + filename, 'r') as fp:
      data = json.load(fp)
  return json_to_text(data)

In [ ]:
artiklite_kaust = "" + "artikklid"

laetud_artiklid = []
with os.scandir(artiklite_kaust) as entries:
  for entry in entries:
    if entry.is_file():
      data = None
      with open(entry, 'r') as fp:
        data = json.load(fp)
      laetud_artiklid.append(json_to_text(data))

print("Lugesin sisse", len(laetud_artiklid), "artikklit.")

Lugesin sisse 80 artikklit.


In [ ]:
# sorteerin, et oleksid kindlas järjestuses võrdlemise eesmärgil
laetud_artiklid.sort(key=lambda x: x.meta.get("filename"))
laetud_artiklid[5]
for i in range(len(laetud_artiklid)):
    if laetud_artiklid[i].meta.get("filename") == 'aja_ml_2002_47.tasak.a023.sol':
        print(i)
laetud_artiklid[6].gold_word_events

6


Layer(name='gold_word_events', attributes=('nertag', 'sentence_ID'), spans=SL[EnvelopingSpan('20.11.2002', [{'nertag': 'O', 'sentence_ID': 0}]),
EnvelopingSpan('Juba', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('aastaid', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('on', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('piimavarumisel', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('kestnud', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('anarhia', [{'nertag': 'B-EVENT', 'sentence_ID': 1}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 1}]),
EnvelopingSpan('Kui', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('vahepeal', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tundubki', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('suuremad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hädad', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('hakkavad', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('mööda', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('saama', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('tuleb', [{'nertag': 'B-EVENT', 'sentence_ID': 2}]),
EnvelopingSpan('uus', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('löök', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 2}]),
EnvelopingSpan('See', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('et', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Rapla', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Dairy', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('on', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('lehmapidajatele', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kuude', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kaupa', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('võlgu', [{'nertag': 'I-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('pälvis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('laiemat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tähelepanu', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('alles', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('siis', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan(',', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kui', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('riik', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('tahtis', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('firmat', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('juhtivale', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Afganistani', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eruohvitserile', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Jaqub', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Haidaryle', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('eriliste', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('teenete', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('eest', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kodakondsuse', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('kinkida', [{'nertag': 'B-EVENT', 'sentence_ID': 3}]),
EnvelopingSpan('.', [{'nertag': 'O', 'sentence_ID': 3}]),
EnvelopingSpan('Mure', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('suuruse', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('võtab', [{'nertag': 'B-EVENT', 'sentence_ID': 4}]),
EnvelopingSpan('oma', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('ilmekas', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('lihtsuses', [{'nertag': 'O', 'sentence_ID': 4}]),
EnvelopingSpan('kok

In [ ]:
for artikkel in laetud_artiklid:
    if len(artikkel.words) != len(artikkel.gold_word_events_synced): # Kihi nimi
        print(artikkel.meta.filename)

# Mudeli treenimine

In [ ]:
from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR

model_dir=DEFAULT_PY3_NER_MODEL_DIR
modelUtil = ModelStorageUtil(model_dir)
nersettings = modelUtil.load_settings()
trainer = NerTrainer(nersettings)

In [ ]:
trains = []
tests = []
kf = KFold(n_splits=10, shuffle=True, random_state=2021)
kf.get_n_splits(laetud_artiklid)
for train_index, test_index in kf.split(laetud_artiklid):
    trains.append(train_index)
    tests.append(test_index)

In [ ]:
acc_list, prec_list, recall_list, f1_score_list = [], [], [], []
true_tags_list, pred_tags_list = [], []
tekstiobjekt = None

In [ ]:
for train_index, test_index in kf.split(laetud_artiklid):
    artiklite_kaust = "" + "artikklid"

    laetud_artiklid = []
    with os.scandir(artiklite_kaust) as entries:
      for entry in entries:
        if entry.is_file():
          data = None
          with open(entry, 'r') as fp:
            data = json.load(fp)
          laetud_artiklid.append(json_to_text(data))

    train = [laetud_artiklid[i] for i in train_index]
    test = [laetud_artiklid[i] for i in test_index]
    
    model_dir=DEFAULT_PY3_NER_MODEL_DIR
    modelUtil = ModelStorageUtil(model_dir)
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train(train, layer='gold_word_events_synced', model_dir="C:\\Users\\Taaniel\\Desktop\\ESTmodel_gold_word_events_synced") # Kihi nimi
    print("Treenitud, nüüd valideerin")
    nertagger = WordLevelNerTagger(model_dir = 'C:\\Users\\Taaniel\\Desktop\\ESTmodel_gold_word_events_synced') # Kihi nimi
    for tekst in test:
        nertagger.tag(tekst)
        
    tags_true = []
    tags_predicted = []
    for tekst in test:
        tags_true.append([predicted.nertag for predicted in tekst.wordner])
        tags_predicted.append([true.nertag for true in tekst.gold_word_events_synced]) # Kihi nimi
    
    true_tags_list.append(tags_true)
    pred_tags_list.append(tags_predicted)
    
    acc_list.append(accuracy_score(tags_true, tags_predicted))
    prec_list.append(precision_score(tags_true, tags_predicted))
    recall_list.append(recall_score(tags_true, tags_predicted))
    f1_score_list.append(f1_score(tags_true, tags_predicted))
    
    print("Validation Accuracy: {}".format(accuracy_score(tags_true, tags_predicted)))
    print("Validation Precision: {}".format(precision_score(tags_true, tags_predicted)))
    print("Validation Recall: {}".format(recall_score(tags_true, tags_predicted)))
    print("Validation F1-Score: {}".format(f1_score(tags_true, tags_predicted)))
    
print("Keskmised")
print("Accuracy", np.mean(acc_list))
print("---", acc_list)
print("Precision", np.mean(prec_list))
print("---", prec_list)
print("Recall", np.mean(recall_list))
print("---", recall_list)
print("F1_score", np.mean(f1_score_list))
print("---", f1_score_list)

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 79933
Seconds required: 0.154

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 18598.407435
Trial #1 (eta = 0.100000): 4010.081761
Trial #2 (eta = 0.200000): 5545.596990
Trial #3 (eta = 0.400000): 10501.190307
Trial #4 (eta = 0.800000): 21479.131963 (worse)
Trial #5 (eta = 0.050000): 3744.710431
Trial #6 (eta = 0.025000): 3909.162938
Trial #7 (eta = 0.012500): 4284.391715
Trial #8 (eta = 0.006250): 4838.733847
Trial #9 (eta = 0.003125): 5567.748310
Trial #10 (eta = 0.001563): 6468.114309
Trial #11 (eta = 0.000781): 7581.530540
Trial #12 (eta = 0.000391): 8944.964682
Trial #

***** Epoch #40 *****
Loss: 201.905454
Improvement ratio: 0.344794
Feature L2-norm: 62.475113
Learning rate (eta): 0.049801
Total number of feature updates: 47520
Seconds required for this iteration: 0.024

***** Epoch #41 *****
Loss: 197.242181
Improvement ratio: 0.330004
Feature L2-norm: 62.925779
Learning rate (eta): 0.049796
Total number of feature updates: 48708
Seconds required for this iteration: 0.021

***** Epoch #42 *****
Loss: 192.396445
Improvement ratio: 0.318850
Feature L2-norm: 63.369347
Learning rate (eta): 0.049791
Total number of feature updates: 49896
Seconds required for this iteration: 0.023

***** Epoch #43 *****
Loss: 188.182287
Improvement ratio: 0.309621
Feature L2-norm: 63.800189
Learning rate (eta): 0.049786
Total number of feature updates: 51084
Seconds required for this iteration: 0.026

***** Epoch #44 *****
Loss: 183.994146
Improvement ratio: 0.294164
Feature L2-norm: 64.222282
Learning rate (eta): 0.049781
Total number of feature updates: 52272
Seconds r

***** Epoch #85 *****
Loss: 97.948731
Improvement ratio: 0.121275
Feature L2-norm: 76.252905
Learning rate (eta): 0.049579
Total number of feature updates: 100980
Seconds required for this iteration: 0.022

***** Epoch #86 *****
Loss: 96.744849
Improvement ratio: 0.120469
Feature L2-norm: 76.465637
Learning rate (eta): 0.049574
Total number of feature updates: 102168
Seconds required for this iteration: 0.021

***** Epoch #87 *****
Loss: 95.854465
Improvement ratio: 0.119658
Feature L2-norm: 76.675904
Learning rate (eta): 0.049569
Total number of feature updates: 103356
Seconds required for this iteration: 0.019

***** Epoch #88 *****
Loss: 94.839368
Improvement ratio: 0.118184
Feature L2-norm: 76.883567
Learning rate (eta): 0.049564
Total number of feature updates: 104544
Seconds required for this iteration: 0.020

***** Epoch #89 *****
Loss: 94.016231
Improvement ratio: 0.113332
Feature L2-norm: 77.088760
Learning rate (eta): 0.049559
Total number of feature updates: 105732
Seconds r

***** Epoch #131 *****
Loss: 67.201242
Improvement ratio: 0.067518
Feature L2-norm: 84.080602
Learning rate (eta): 0.049353
Total number of feature updates: 155628
Seconds required for this iteration: 0.023

***** Epoch #132 *****
Loss: 66.763412
Improvement ratio: 0.070172
Feature L2-norm: 84.217548
Learning rate (eta): 0.049349
Total number of feature updates: 156816
Seconds required for this iteration: 0.020

***** Epoch #133 *****
Loss: 66.330122
Improvement ratio: 0.067627
Feature L2-norm: 84.353166
Learning rate (eta): 0.049344
Total number of feature updates: 158004
Seconds required for this iteration: 0.020

***** Epoch #134 *****
Loss: 66.042187
Improvement ratio: 0.065141
Feature L2-norm: 84.488158
Learning rate (eta): 0.049339
Total number of feature updates: 159192
Seconds required for this iteration: 0.020

***** Epoch #135 *****
Loss: 65.658899
Improvement ratio: 0.065710
Feature L2-norm: 84.621584
Learning rate (eta): 0.049334
Total number of feature updates: 160380
Seco

***** Epoch #179 *****
Loss: 52.699693
Improvement ratio: 0.043078
Feature L2-norm: 89.662858
Learning rate (eta): 0.049121
Total number of feature updates: 212652
Seconds required for this iteration: 0.019

***** Epoch #180 *****
Loss: 52.464593
Improvement ratio: 0.042525
Feature L2-norm: 89.761888
Learning rate (eta): 0.049116
Total number of feature updates: 213840
Seconds required for this iteration: 0.021

***** Epoch #181 *****
Loss: 52.244601
Improvement ratio: 0.041965
Feature L2-norm: 89.860241
Learning rate (eta): 0.049111
Total number of feature updates: 215028
Seconds required for this iteration: 0.020

***** Epoch #182 *****
Loss: 52.056441
Improvement ratio: 0.042975
Feature L2-norm: 89.958084
Learning rate (eta): 0.049106
Total number of feature updates: 216216
Seconds required for this iteration: 0.020

***** Epoch #183 *****
Loss: 51.795089
Improvement ratio: 0.042293
Feature L2-norm: 90.055437
Learning rate (eta): 0.049101
Total number of feature updates: 217404
Seco

...

***** Epoch #808 *****
Loss: 25.218760
Improvement ratio: 0.002856
Feature L2-norm: 114.228290
Learning rate (eta): 0.046262
Total number of feature updates: 961520
Seconds required for this iteration: 0.022

***** Epoch #809 *****
Loss: 25.212418
Improvement ratio: 0.002774
Feature L2-norm: 114.245750
Learning rate (eta): 0.046258
Total number of feature updates: 962710
Seconds required for this iteration: 0.023

***** Epoch #810 *****
Loss: 25.200484
Improvement ratio: 0.002863
Feature L2-norm: 114.263178
Learning rate (eta): 0.046253
Total number of feature updates: 963900
Seconds required for this iteration: 0.023

***** Epoch #811 *****
Loss: 25.198182
Improvement ratio: 0.002812
Feature L2-norm: 114.280566
Learning rate (eta): 0.046249
Total number of feature updates: 965090
Seconds required for this iteration: 0.026

***** Epoch #812 *****
Loss: 25.187690
Improvement ratio: 0.002802
Feature L2-norm: 114.297944
Learning rate (eta): 0.046245
Total number of feature updates: 966280

***** Epoch #851 *****
Loss: 24.929514
Improvement ratio: 0.002562
Feature L2-norm: 114.953851
Learning rate (eta): 0.046079
Total number of feature updates: 1012690
Seconds required for this iteration: 0.025

***** Epoch #852 *****
Loss: 24.922362
Improvement ratio: 0.002553
Feature L2-norm: 114.970149
Learning rate (eta): 0.046074
Total number of feature updates: 1013880
Seconds required for this iteration: 0.024

***** Epoch #853 *****
Loss: 24.918560
Improvement ratio: 0.001956
Feature L2-norm: 114.986427
Learning rate (eta): 0.046070
Total number of feature updates: 1015070
Seconds required for this iteration: 0.024

***** Epoch #854 *****
Loss: 24.906419
Improvement ratio: 0.002775
Feature L2-norm: 115.002651
Learning rate (eta): 0.046066
Total number of feature updates: 1016260
Seconds required for this iteration: 0.024

***** Epoch #855 *****
Loss: 24.901319
Improvement ratio: 0.002657
Feature L2-norm: 115.018867
Learning rate (eta): 0.046062
Total number of feature updates: 10

***** Epoch #894 *****
Loss: 24.677524
Improvement ratio: 0.002240
Feature L2-norm: 115.631821
Learning rate (eta): 0.045897
Total number of feature updates: 1063860
Seconds required for this iteration: 0.023

***** Epoch #895 *****
Loss: 24.665482
Improvement ratio: 0.002513
Feature L2-norm: 115.647054
Learning rate (eta): 0.045893
Total number of feature updates: 1065050
Seconds required for this iteration: 0.025

***** Epoch #896 *****
Loss: 24.664197
Improvement ratio: 0.001998
Feature L2-norm: 115.662274
Learning rate (eta): 0.045888
Total number of feature updates: 1066240
Seconds required for this iteration: 0.022

***** Epoch #897 *****
Loss: 24.660566
Improvement ratio: 0.002374
Feature L2-norm: 115.677451
Learning rate (eta): 0.045884
Total number of feature updates: 1067430
Seconds required for this iteration: 0.026

***** Epoch #898 *****
Loss: 24.654102
Improvement ratio: 0.002216
Feature L2-norm: 115.692623
Learning rate (eta): 0.045880
Total number of feature updates: 10

***** Epoch #935 *****
Loss: 24.462783
Improvement ratio: 0.001666
Feature L2-norm: 116.238101
Learning rate (eta): 0.045725
Total number of feature updates: 1112650
Seconds required for this iteration: 0.025

***** Epoch #936 *****
Loss: 24.454497
Improvement ratio: 0.002253
Feature L2-norm: 116.252421
Learning rate (eta): 0.045721
Total number of feature updates: 1113840
Seconds required for this iteration: 0.026

***** Epoch #937 *****
Loss: 24.448140
Improvement ratio: 0.002175
Feature L2-norm: 116.266723
Learning rate (eta): 0.045716
Total number of feature updates: 1115030
Seconds required for this iteration: 0.024

***** Epoch #938 *****
Loss: 24.451033
Improvement ratio: 0.001755
Feature L2-norm: 116.281005
Learning rate (eta): 0.045712
Total number of feature updates: 1116220
Seconds required for this iteration: 0.026

***** Epoch #939 *****
Loss: 24.443623
Improvement ratio: 0.002008
Feature L2-norm: 116.295276
Learning rate (eta): 0.045708
Total number of feature updates: 11

***** Epoch #977 *****
Loss: 24.269100
Improvement ratio: 0.001419
Feature L2-norm: 116.822188
Learning rate (eta): 0.045550
Total number of feature updates: 1162630
Seconds required for this iteration: 0.022

***** Epoch #978 *****
Loss: 24.266117
Improvement ratio: 0.001755
Feature L2-norm: 116.835663
Learning rate (eta): 0.045546
Total number of feature updates: 1163820
Seconds required for this iteration: 0.023

***** Epoch #979 *****
Loss: 24.259692
Improvement ratio: 0.001649
Feature L2-norm: 116.849122
Learning rate (eta): 0.045541
Total number of feature updates: 1165010
Seconds required for this iteration: 0.024

***** Epoch #980 *****
Loss: 24.251035
Improvement ratio: 0.002032
Feature L2-norm: 116.862555
Learning rate (eta): 0.045537
Total number of feature updates: 1166200
Seconds required for this iteration: 0.025

***** Epoch #981 *****
Loss: 24.249488
Improvement ratio: 0.001902
Feature L2-norm: 116.875979
Learning rate (eta): 0.045533
Total number of feature updates: 11

In [ ]:
all_pred_tags = []
all_true_tags = []
for epoch in range(len(pred_tags_list)):
    all_pred_tags.extend(pred_tags_list[epoch])
    all_true_tags.extend(true_tags_list[epoch])

print("Kasutades kõiki ennustatud ja kuldstandardi märgendeid")
print("Accuracy: {}".format(accuracy_score(all_pred_tags, all_true_tags)))
print("Precision: {}".format(precision_score(all_pred_tags, all_true_tags)))
print("Recall: {}".format(recall_score(all_pred_tags, all_true_tags)))
print("F1-Score: {}".format(f1_score(all_pred_tags, all_true_tags)))

Kasutades kõiki ennustatud ja kuldstandardi märgendeid
Accuracy: 0.9422581665386527
Precision: 0.8735301802978834
Recall: 0.8007185628742515
F1-Score: 0.8355411147213195


In [ ]:
from nervaluate import Evaluator
# gold_word_events = 'EVENT'
# _w_classes = 'EVENT_ASPECTUAL', 'EVENT_I_ACTION','EVENT_I_STATE','EVENT_MODAL','EVENT_OCCURRENCE','EVENT_PERCEPTION','EVENT_REPORTING','EVENT_STATE','EVENT_ASPECTUAL','EVENT_I_ACTION','EVENT_I_STATE','EVENT_OCCURRENCE','EVENT_STATE'
# _only_main = 'EVENT_ASPECTUAL','EVENT_I_ACTION','EVENT_I_STATE','EVENT_MODAL','EVENT_OCCURRENCE','EVENT_PERCEPTION','EVENT_REPORTING','EVENT_STATE'
evaluator = Evaluator(all_true_tags, all_pred_tags, tags=['EVENT'], loader="list")
results, results_by_tag = evaluator.evaluate()

In [ ]:
results

{'ent_type': {'correct': 3467,
  'incorrect': 0,
  'partial': 0,
  'missed': 360,
  'spurious': 708,
  'possible': 3827,
  'actual': 4175,
  'precision': 0.8304191616766468,
  'recall': 0.9059315390645414,
  'f1': 0.8665333666583356},
 'partial': {'correct': 3343,
  'incorrect': 0,
  'partial': 124,
  'missed': 360,
  'spurious': 708,
  'possible': 3827,
  'actual': 4175,
  'precision': 0.8155688622754491,
  'recall': 0.8897308596812125,
  'f1': 0.8510372406898274},
 'strict': {'correct': 3343,
  'incorrect': 124,
  'partial': 0,
  'missed': 360,
  'spurious': 708,
  'possible': 3827,
  'actual': 4175,
  'precision': 0.8007185628742515,
  'recall': 0.8735301802978834,
  'f1': 0.8355411147213195},
 'exact': {'correct': 3343,
  'incorrect': 124,
  'partial': 0,
  'missed': 360,
  'spurious': 708,
  'possible': 3827,
  'actual': 4175,
  'precision': 0.8007185628742515,
  'recall': 0.8735301802978834,
  'f1': 0.8355411147213195}}

In [ ]:
results_by_tag

{'EVENT': {'ent_type': {'correct': 3467,
   'incorrect': 0,
   'partial': 0,
   'missed': 360,
   'spurious': 708,
   'possible': 3827,
   'actual': 4175,
   'precision': 0.8304191616766468,
   'recall': 0.9059315390645414,
   'f1': 0.8665333666583356},
  'partial': {'correct': 3343,
   'incorrect': 0,
   'partial': 124,
   'missed': 360,
   'spurious': 708,
   'possible': 3827,
   'actual': 4175,
   'precision': 0.8155688622754491,
   'recall': 0.8897308596812125,
   'f1': 0.8510372406898274},
  'strict': {'correct': 3343,
   'incorrect': 124,
   'partial': 0,
   'missed': 360,
   'spurious': 708,
   'possible': 3827,
   'actual': 4175,
   'precision': 0.8007185628742515,
   'recall': 0.8735301802978834,
   'f1': 0.8355411147213195},
  'exact': {'correct': 3343,
   'incorrect': 124,
   'partial': 0,
   'missed': 360,
   'spurious': 708,
   'possible': 3827,
   'actual': 4175,
   'precision': 0.8007185628742515,
   'recall': 0.8735301802978834,
   'f1': 0.8355411147213195}}}

# Tulemused

## gold_word_events_synced
Keskmised  
Accuracy 0.9422058404227822  
> [0.943170964660936, 0.9571725571725572, 0.9378190255220418, 0.9362092703074805, 0.9357038480272771, 0.9410562180579216, 0.952529994783516, 0.937533948940793, 0.9323858742463393, 0.9484767025089605]  

Precision 0.8007434718537093  
> [0.8717948717948718, 0.8885941644562334, 0.8696741854636592, 0.865546218487395, 0.8382352941176471, 0.8687050359712231, 0.9154078549848943, 0.8584615384615385, 0.8676844783715013, 0.8941504178272981]  

Recall 0.8738254059936261  
> [0.8173076923076923, 0.8524173027989822, 0.7977011494252874, 0.7610837438423645, 0.7808219178082192, 0.8104026845637584, 0.8347107438016529, 0.7859154929577464, 0.7645739910313901, 0.8025]  

F1_score 0.8355458957907145  
> [0.8436724565756825, 0.8701298701298701, 0.8321342925659472, 0.8099606815203145, 0.8085106382978724, 0.8385416666666666, 0.873198847262248, 0.8205882352941176, 0.8128724672228843, 0.8458498023715415]  

**Kasutades kõiki ennustatud ja kuldstandardi märgendeid**  
Accuracy: 0.9422581665386527  
Precision: 0.8735301802978834  
Recall: 0.8007185628742515  
F1-Score: 0.8355411147213195  

### Nervaluate
{'ent_type': {'correct': 3467,  
&emsp;'incorrect': 0,  
&emsp;'partial': 0,  
&emsp;'missed': 360,  
&emsp;'spurious': 708,  
&emsp;'possible': 3827,  
&emsp;'actual': 4175,  
&emsp;'precision': 0.8304191616766468,  
&emsp;'recall': 0.9059315390645414,  
&emsp;'f1': 0.8665333666583356},  
 'partial': {'correct': 3343,  
&emsp;'incorrect': 0,  
&emsp;'partial': 124,  
&emsp;'missed': 360,  
&emsp;'spurious': 708,  
&emsp;'possible': 3827,  
&emsp;'actual': 4175,  
&emsp;'precision': 0.8155688622754491,  
&emsp;'recall': 0.8897308596812125,  
&emsp;'f1': 0.8510372406898274},  
 'strict': {'correct': 3343,  
&emsp;'incorrect': 124,  
&emsp;'partial': 0,  
&emsp;'missed': 360,  
&emsp;'spurious': 708,  
&emsp;'possible': 3827,  
&emsp;'actual': 4175,  
&emsp;'precision': 0.8007185628742515,  
&emsp;'recall': 0.8735301802978834,  
&emsp;'f1': 0.8355411147213195},  
 'exact': {'correct': 3343,  
&emsp;'incorrect': 124,  
&emsp;'partial': 0,  
&emsp;'missed': 360,  
&emsp;'spurious': 708,  
&emsp;'possible': 3827,  
&emsp;'actual': 4175,  
&emsp;'precision': 0.8007185628742515,  
&emsp;'recall': 0.8735301802978834,  
&emsp;'f1': 0.8355411147213195}}  

## gold_word_events_w_classes_synced
Keskmised  
Accuracy 0.912018532738772  
> [0.9178605539637058, 0.9251559251559252, 0.9006960556844548, 0.9040844424047728, 0.9147588894301023, 0.9189097103918228, 0.9191444966092853, 0.8946224877783813, 0.9078380706287683, 0.9171146953405018]

Precision 0.634556847289408  
> [0.6714628297362111, 0.6590330788804071, 0.5986238532110092, 0.5922330097087378, 0.6438356164383562, 0.686046511627907, 0.6446280991735537, 0.5854341736694678, 0.6367713004484304, 0.6275]

Recall 0.7114466319584067 
> [0.7349081364829396, 0.6981132075471698, 0.6832460732984293, 0.7011494252873564, 0.7253086419753086, 0.7577981651376147, 0.7289719626168224, 0.6510903426791277, 0.7208121827411168, 0.7130681818181818]

F1_score 0.6706772832626524
> [0.7017543859649122, 0.6780104712041883, 0.6381418092909537, 0.6421052631578947, 0.6821480406386067, 0.7201394943330427, 0.6842105263157894, 0.6165191740412979, 0.6761904761904762, 0.6675531914893618]

**Kasutades kõiki ennustatud ja kuldstandardi märgendeid**  
Accuracy: 0.9125288031446257  
Precision: 0.7140946777213159  
Recall: 0.6370794559770938  
F1-Score: 0.6733921815889029  

### Nervaluate
{'ent_type': {'correct': 2767,  
&emsp;'incorrect': 659,  
&emsp;'partial': 0,  
&emsp;'missed': 313,  
&emsp;'spurious': 765,  
&emsp;'possible': 3739,  
&emsp;'actual': 4191,  
&emsp;'precision': 0.66022429014555,  
&emsp;'recall': 0.7400374431666221,  
&emsp;'f1': 0.6978562421185373},  
 'partial': {'correct': 3305,  
&emsp;'incorrect': 0,  
&emsp;'partial': 121,  
&emsp;'missed': 313,  
&emsp;'spurious': 765,  
&emsp;'possible': 3739,  
&emsp;'actual': 4191,  
&emsp;'precision': 0.803030303030303,  
&emsp;'recall': 0.9001069804760631,  
&emsp;'f1': 0.8488020176544766},  
 'strict': {'correct': 2670,  
&emsp;'incorrect': 756,  
&emsp;'partial': 0,  
&emsp;'missed': 313,  
&emsp;'spurious': 765,  
&emsp;'possible': 3739,  
&emsp;'actual': 4191,  
&emsp;'precision': 0.6370794559770938,  
&emsp;'recall': 0.7140946777213159,  
&emsp;'f1': 0.6733921815889029},  
 'exact': {'correct': 3305,  
&emsp;'incorrect': 121,  
&emsp;'partial': 0,  
&emsp;'missed': 313,  
&emsp;'spurious': 765,  
&emsp;'possible': 3739,  
&emsp;'actual': 4191,  
&emsp;'precision': 0.7885946074922453,  
&emsp;'recall': 0.8839261834715164,  
&emsp;'f1': 0.8335435056746533}}  

### Nervaluate by tag
{'EVENT_ASPECTUAL': {'ent_type': {'correct': 88,  
&emsp;&emsp;'incorrect': 22,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 20,  
&emsp;&emsp;'possible': 116,  
&emsp;&emsp;'actual': 130,  
&emsp;&emsp;'precision': 0.676923076923077,  
&emsp;&emsp;'recall': 0.7586206896551724,  
&emsp;&emsp;'f1': 0.7154471544715447},  
&emsp;'partial': {'correct': 110,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 20,  
&emsp;&emsp;'possible': 116,  
&emsp;&emsp;'actual': 130,  
&emsp;&emsp;'precision': 0.8461538461538461,  
&emsp;&emsp;'recall': 0.9482758620689655,  
&emsp;&emsp;'f1': 0.894308943089431},  
&emsp;'strict': {'correct': 88,  
&emsp;&emsp;'incorrect': 22,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 20,  
&emsp;&emsp;'possible': 116,  
&emsp;&emsp;'actual': 130,  
&emsp;&emsp;'precision': 0.676923076923077,  
&emsp;&emsp;'recall': 0.7586206896551724,  
&emsp;&emsp;'f1': 0.7154471544715447},  
&emsp;'exact': {'correct': 110,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 20,  
&emsp;&emsp;'possible': 116,  
&emsp;&emsp;'actual': 130,  
&emsp;&emsp;'precision': 0.8461538461538461,  
&emsp;&emsp;'recall': 0.9482758620689655,  
&emsp;&emsp;'f1': 0.894308943089431}},  
 'EVENT_I_ACTION': {'ent_type': {'correct': 172,  
&emsp;&emsp;'incorrect': 154,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 26,  
&emsp;&emsp;'possible': 328,  
&emsp;&emsp;'actual': 352,  
&emsp;&emsp;'precision': 0.48863636363636365,  
&emsp;&emsp;'recall': 0.524390243902439,  
&emsp;&emsp;'f1': 0.5058823529411763},  
&emsp;'partial': {'correct': 326,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 26,  
&emsp;&emsp;'possible': 328,  
&emsp;&emsp;'actual': 352,  
&emsp;&emsp;'precision': 0.9261363636363636,  
&emsp;&emsp;'recall': 0.9939024390243902,  
&emsp;&emsp;'f1': 0.9588235294117649},  
&emsp;'strict': {'correct': 172,  
&emsp;&emsp;'incorrect': 154,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 26,  
&emsp;&emsp;'possible': 328,  
&emsp;&emsp;'actual': 352,  
&emsp;&emsp;'precision': 0.48863636363636365,  
&emsp;&emsp;'recall': 0.524390243902439,  
&emsp;&emsp;'f1': 0.5058823529411763},  
&emsp;'exact': {'correct': 326,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 26,  
&emsp;&emsp;'possible': 328,  
&emsp;&emsp;'actual': 352,  
&emsp;&emsp;'precision': 0.9261363636363636,  
&emsp;&emsp;'recall': 0.9939024390243902,  
&emsp;&emsp;'f1': 0.9588235294117649}},  
 'EVENT_I_STATE': {'ent_type': {'correct': 200,  
&emsp;&emsp;'incorrect': 80,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 8,  
&emsp;&emsp;'spurious': 78,  
&emsp;&emsp;'possible': 288,  
&emsp;&emsp;'actual': 358,  
&emsp;&emsp;'precision': 0.5586592178770949,  
&emsp;&emsp;'recall': 0.6944444444444444,  
&emsp;&emsp;'f1': 0.6191950464396285},  
&emsp;'partial': {'correct': 280,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 8,  
&emsp;&emsp;'spurious': 78,  
&emsp;&emsp;'possible': 288,  
&emsp;&emsp;'actual': 358,  
&emsp;&emsp;'precision': 0.7821229050279329,  
&emsp;&emsp;'recall': 0.9722222222222222,  
&emsp;&emsp;'f1': 0.8668730650154798},  
&emsp;'strict': {'correct': 200,  
&emsp;&emsp;'incorrect': 80,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 8,  
&emsp;&emsp;'spurious': 78,  
&emsp;&emsp;'possible': 288,  
&emsp;&emsp;'actual': 358,  
&emsp;&emsp;'precision': 0.5586592178770949,  
&emsp;&emsp;'recall': 0.6944444444444444,  
&emsp;&emsp;'f1': 0.6191950464396285},  
&emsp;'exact': {'correct': 280,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 8,  
&emsp;&emsp;'spurious': 78,  
&emsp;&emsp;'possible': 288,  
&emsp;&emsp;'actual': 358,  
&emsp;&emsp;'precision': 0.7821229050279329,  
&emsp;&emsp;'recall': 0.9722222222222222,  
&emsp;&emsp;'f1': 0.8668730650154798}},  
 'EVENT_MODAL': {'ent_type': {'correct': 131,  
&emsp;&emsp;'incorrect': 22,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 154,  
&emsp;&emsp;'actual': 155,  
&emsp;&emsp;'precision': 0.8451612903225807,  
&emsp;&emsp;'recall': 0.8506493506493507,  
&emsp;&emsp;'f1': 0.8478964401294499},  
&emsp;'partial': {'correct': 152,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 1,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 154,  
&emsp;&emsp;'actual': 155,  
&emsp;&emsp;'precision': 0.9838709677419355,  
&emsp;&emsp;'recall': 0.9902597402597403,  
&emsp;&emsp;'f1': 0.9870550161812298},  
&emsp;'strict': {'correct': 131,  
&emsp;&emsp;'incorrect': 22,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 154,  
&emsp;&emsp;'actual': 155,  
&emsp;&emsp;'precision': 0.8451612903225807,  
&emsp;&emsp;'recall': 0.8506493506493507,  
&emsp;&emsp;'f1': 0.8478964401294499},  
&emsp;'exact': {'correct': 152,  
&emsp;&emsp;'incorrect': 1,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 154,  
&emsp;&emsp;'actual': 155,  
&emsp;&emsp;'precision': 0.9806451612903225,  
&emsp;&emsp;'recall': 0.987012987012987,  
&emsp;&emsp;'f1': 0.9838187702265372}},  
 'EVENT_OCCURRENCE': {'ent_type': {'correct': 3384,  
&emsp;&emsp;'incorrect': 782,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 370,  
&emsp;&emsp;'spurious': 974,  
&emsp;&emsp;'possible': 4536,  
&emsp;&emsp;'actual': 5140,  
&emsp;&emsp;'precision': 0.6583657587548638,  
&emsp;&emsp;'recall': 0.746031746031746,  
&emsp;&emsp;'f1': 0.6994625878462175},  
&emsp;'partial': {'correct': 4082,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 84,  
&emsp;&emsp;'missed': 370,  
&emsp;&emsp;'spurious': 974,  
&emsp;&emsp;'possible': 4536,  
&emsp;&emsp;'actual': 5140,  
&emsp;&emsp;'precision': 0.8023346303501946,  
&emsp;&emsp;'recall': 0.9091710758377425,  
&emsp;&emsp;'f1': 0.8524183546920215},  
&emsp;'strict': {'correct': 3324,  
&emsp;&emsp;'incorrect': 842,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 370,  
&emsp;&emsp;'spurious': 974,  
&emsp;&emsp;'possible': 4536,  
&emsp;&emsp;'actual': 5140,  
&emsp;&emsp;'precision': 0.6466926070038911,  
&emsp;&emsp;'recall': 0.7328042328042328,  
&emsp;&emsp;'f1': 0.6870607689127739},  
&emsp;'exact': {'correct': 4082,  
&emsp;&emsp;'incorrect': 84,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 370,  
&emsp;&emsp;'spurious': 974,  
&emsp;&emsp;'possible': 4536,  
&emsp;&emsp;'actual': 5140,  
&emsp;&emsp;'precision': 0.7941634241245136,  
&emsp;&emsp;'recall': 0.8999118165784833,  
&emsp;&emsp;'f1': 0.843737081438611}},  
 'EVENT_PERCEPTION': {'ent_type': {'correct': 0,  
&emsp;&emsp;'incorrect': 2,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 2,  
&emsp;&emsp;'actual': 2,  
&emsp;&emsp;'precision': 0.0,  
&emsp;&emsp;'recall': 0.0,  
&emsp;&emsp;'f1': 0},  
&emsp;'partial': {'correct': 2,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 2,  
&emsp;&emsp;'actual': 2,  
&emsp;&emsp;'precision': 1.0,  
&emsp;&emsp;'recall': 1.0,  
&emsp;&emsp;'f1': 1.0},  
&emsp;'strict': {'correct': 0,  
&emsp;&emsp;'incorrect': 2,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 2,  
&emsp;&emsp;'actual': 2,  
&emsp;&emsp;'precision': 0.0,  
&emsp;&emsp;'recall': 0.0,  
&emsp;&emsp;'f1': 0},  
&emsp;'exact': {'correct': 2,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 2,  
&emsp;&emsp;'actual': 2,  
&emsp;&emsp;'precision': 1.0,  
&emsp;&emsp;'recall': 1.0,  
&emsp;&emsp;'f1': 1.0}},  
 'EVENT_REPORTING': {'ent_type': {'correct': 223,  
&emsp;&emsp;'incorrect': 31,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 255,  
&emsp;&emsp;'actual': 258,  
&emsp;&emsp;'precision': 0.8643410852713178,  
&emsp;&emsp;'recall': 0.8745098039215686,  
&emsp;&emsp;'f1': 0.8693957115009746},  
&emsp;'partial': {'correct': 254,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 255,  
&emsp;&emsp;'actual': 258,  
&emsp;&emsp;'precision': 0.9844961240310077,  
&emsp;&emsp;'recall': 0.996078431372549,  
&emsp;&emsp;'f1': 0.9902534113060429},  
&emsp;'strict': {'correct': 223,  
&emsp;&emsp;'incorrect': 31,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 255,  
&emsp;&emsp;'actual': 258,  
&emsp;&emsp;'precision': 0.8643410852713178,  
&emsp;&emsp;'recall': 0.8745098039215686,  
&emsp;&emsp;'f1': 0.8693957115009746},  
&emsp;'exact': {'correct': 254,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 255,  
&emsp;&emsp;'actual': 258,  
&emsp;&emsp;'precision': 0.9844961240310077,  
&emsp;&emsp;'recall': 0.996078431372549,  
&emsp;&emsp;'f1': 0.9902534113060429}},  
 'EVENT_STATE': {'ent_type': {'correct': 982,  
&emsp;&emsp;'incorrect': 170,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 236,  
&emsp;&emsp;'spurious': 420,  
&emsp;&emsp;'possible': 1388,  
&emsp;&emsp;'actual': 1572,  
&emsp;&emsp;'precision': 0.6246819338422391,  
&emsp;&emsp;'recall': 0.707492795389049,  
&emsp;&emsp;'f1': 0.6635135135135135},  
&emsp;'partial': {'correct': 996,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 156,  
&emsp;&emsp;'missed': 236,  
&emsp;&emsp;'spurious': 420,  
&emsp;&emsp;'possible': 1388,  
&emsp;&emsp;'actual': 1572,  
&emsp;&emsp;'precision': 0.683206106870229,  
&emsp;&emsp;'recall': 0.7737752161383286,  
&emsp;&emsp;'f1': 0.7256756756756757},  
&emsp;'strict': {'correct': 848,  
&emsp;&emsp;'incorrect': 304,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 236,  
&emsp;&emsp;'spurious': 420,  
&emsp;&emsp;'possible': 1388,  
&emsp;&emsp;'actual': 1572,  
&emsp;&emsp;'precision': 0.539440203562341,  
&emsp;&emsp;'recall': 0.6109510086455331,  
&emsp;&emsp;'f1': 0.5729729729729729},  
&emsp;'exact': {'correct': 996,  
&emsp;&emsp;'incorrect': 156,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 236,  
&emsp;&emsp;'spurious': 420,  
&emsp;&emsp;'possible': 1388,  
&emsp;&emsp;'actual': 1572,  
&emsp;&emsp;'precision': 0.6335877862595419,  
&emsp;&emsp;'recall': 0.7175792507204611,  
&emsp;&emsp;'f1': 0.672972972972973}}}

## gold_word_events_only_main_synced
Keskmised  
Accuracy 0.9270340587987901  
> [0.9297994269340975, 0.9397089397089398, 0.917401392111369, 0.9219825608077099, 0.9313200194836824, 0.9308347529812606, 0.9316640584246219, 0.9071156979902227, 0.9259259259259259, 0.9345878136200717]

Precision 0.6774795206644477  
> [0.7103274559193955, 0.7052631578947368, 0.6402877697841727, 0.6472081218274112, 0.6914285714285714, 0.7332185886402753, 0.6762177650429799, 0.5994152046783626, 0.6857142857142857, 0.6857142857142857]

Recall 0.7407600115675589
> [0.7601078167115903, 0.7243243243243244, 0.7158176943699732, 0.7391304347826086, 0.7610062893081762, 0.7874306839186691, 0.7539936102236422, 0.6612903225806451, 0.7480519480519481, 0.7564469914040115]

F1_score 0.7075721298667477
> [0.734375, 0.7146666666666667, 0.6759493670886076, 0.6901217861975643, 0.7245508982035929, 0.7593582887700535, 0.7129909365558913, 0.6288343558282209, 0.7155279503105589, 0.7193460490463215]

**Kasutades kõiki ennustatud ja kuldstandardi märgendeid**  
Accuracy: 0.9275290290516424  
Precision: 0.7436734693877551  
Recall: 0.6806973848069738  
F1-Score: 0.7107932379713914  

### Nervaluate
{'ent_type': {'correct': 2733,  
&emsp;'incorrect': 620,  
&emsp;'partial': 0,  
&emsp;'missed': 322,  
&emsp;'spurious': 662,  
&emsp;'possible': 3675,  
&emsp;'actual': 4015,  
&emsp;'precision': 0.6806973848069738,  
&emsp;'recall': 0.7436734693877551,  
&emsp;'f1': 0.7107932379713914},  
 'partial': {'correct': 3353,  
&emsp;'incorrect': 0,  
&emsp;'partial': 0,  
&emsp;'missed': 322,  
&emsp;'spurious': 662,  
&emsp;'possible': 3675,  
&emsp;'actual': 4015,  
&emsp;'precision': 0.8351183063511831,  
&emsp;'recall': 0.9123809523809524,  
&emsp;'f1': 0.8720416124837451},  
 'strict': {'correct': 2733,  
&emsp;'incorrect': 620,  
&emsp;'partial': 0,  
&emsp;'missed': 322,  
&emsp;'spurious': 662,  
&emsp;'possible': 3675,  
&emsp;'actual': 4015,  
&emsp;'precision': 0.6806973848069738,  
&emsp;'recall': 0.7436734693877551,  
&emsp;'f1': 0.7107932379713914},  
 'exact': {'correct': 3353,  
&emsp;'incorrect': 0,  
&emsp;'partial': 0,  
&emsp;'missed': 322,  
&emsp;'spurious': 662,  
&emsp;'possible': 3675,  
&emsp;'actual': 4015,  
&emsp;'precision': 0.8351183063511831,  
&emsp;'recall': 0.9123809523809524,  
&emsp;'f1': 0.8720416124837451}}  

### Nervaluate by tag
{'EVENT_ASPECTUAL': {'ent_type': {'correct': 92,  
&emsp;&emsp;'incorrect': 26,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 18,  
&emsp;&emsp;'possible': 124,  
&emsp;&emsp;'actual': 136,  
&emsp;&emsp;'precision': 0.6764705882352942,  
&emsp;&emsp;'recall': 0.7419354838709677,  
&emsp;&emsp;'f1': 0.7076923076923077},  
&emsp;'partial': {'correct': 118,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 18,  
&emsp;&emsp;'possible': 124,  
&emsp;&emsp;'actual': 136,  
&emsp;&emsp;'precision': 0.8676470588235294,  
&emsp;&emsp;'recall': 0.9516129032258065,  
&emsp;&emsp;'f1': 0.9076923076923077},  
&emsp;'strict': {'correct': 92,  
&emsp;&emsp;'incorrect': 26,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 18,  
&emsp;&emsp;'possible': 124,  
&emsp;&emsp;'actual': 136,  
&emsp;&emsp;'precision': 0.6764705882352942,  
&emsp;&emsp;'recall': 0.7419354838709677,  
&emsp;&emsp;'f1': 0.7076923076923077},  
&emsp;'exact': {'correct': 118,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 6,  
&emsp;&emsp;'spurious': 18,  
&emsp;&emsp;'possible': 124,  
&emsp;&emsp;'actual': 136,  
&emsp;&emsp;'precision': 0.8676470588235294,  
&emsp;&emsp;'recall': 0.9516129032258065,  
&emsp;&emsp;'f1': 0.9076923076923077}},  
 'EVENT_I_ACTION': {'ent_type': {'correct': 170,  
&emsp;&emsp;'incorrect': 146,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 14,  
&emsp;&emsp;'possible': 318,  
&emsp;&emsp;'actual': 330,  
&emsp;&emsp;'precision': 0.5151515151515151,  
&emsp;&emsp;'recall': 0.5345911949685535,  
&emsp;&emsp;'f1': 0.5246913580246912},  
&emsp;'partial': {'correct': 316,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 14,  
&emsp;&emsp;'possible': 318,  
&emsp;&emsp;'actual': 330,  
&emsp;&emsp;'precision': 0.9575757575757575,  
&emsp;&emsp;'recall': 0.9937106918238994,  
&emsp;&emsp;'f1': 0.9753086419753086},  
&emsp;'strict': {'correct': 170,  
&emsp;&emsp;'incorrect': 146,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 14,  
&emsp;&emsp;'possible': 318,  
&emsp;&emsp;'actual': 330,  
&emsp;&emsp;'precision': 0.5151515151515151,  
&emsp;&emsp;'recall': 0.5345911949685535,  
&emsp;&emsp;'f1': 0.5246913580246912},  
&emsp;'exact': {'correct': 316,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 2,  
&emsp;&emsp;'spurious': 14,  
&emsp;&emsp;'possible': 318,  
&emsp;&emsp;'actual': 330,  
&emsp;&emsp;'precision': 0.9575757575757575,  
&emsp;&emsp;'recall': 0.9937106918238994,  
&emsp;&emsp;'f1': 0.9753086419753086}},  
 'EVENT_I_STATE': {'ent_type': {'correct': 196,  
&emsp;&emsp;'incorrect': 72,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 14,  
&emsp;&emsp;'spurious': 70,  
&emsp;&emsp;'possible': 282,  
&emsp;&emsp;'actual': 338,  
&emsp;&emsp;'precision': 0.5798816568047337,  
&emsp;&emsp;'recall': 0.6950354609929078,  
&emsp;&emsp;'f1': 0.632258064516129},  
&emsp;'partial': {'correct': 268,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 14,  
&emsp;&emsp;'spurious': 70,  
&emsp;&emsp;'possible': 282,  
&emsp;&emsp;'actual': 338,  
&emsp;&emsp;'precision': 0.7928994082840237,  
&emsp;&emsp;'recall': 0.950354609929078,  
&emsp;&emsp;'f1': 0.8645161290322582},  
&emsp;'strict': {'correct': 196,  
&emsp;&emsp;'incorrect': 72,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 14,  
&emsp;&emsp;'spurious': 70,  
&emsp;&emsp;'possible': 282,  
&emsp;&emsp;'actual': 338,  
&emsp;&emsp;'precision': 0.5798816568047337,  
&emsp;&emsp;'recall': 0.6950354609929078,  
&emsp;&emsp;'f1': 0.632258064516129},  
&emsp;'exact': {'correct': 268,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 14,  
&emsp;&emsp;'spurious': 70,  
&emsp;&emsp;'possible': 282,  
&emsp;&emsp;'actual': 338,  
&emsp;&emsp;'precision': 0.7928994082840237,  
&emsp;&emsp;'recall': 0.950354609929078,  
&emsp;&emsp;'f1': 0.8645161290322582}},  
 'EVENT_MODAL': {'ent_type': {'correct': 131,  
&emsp;&emsp;'incorrect': 23,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 155,  
&emsp;&emsp;'actual': 156,  
&emsp;&emsp;'precision': 0.8397435897435898,  
&emsp;&emsp;'recall': 0.8451612903225807,  
&emsp;&emsp;'f1': 0.842443729903537},  
&emsp;'partial': {'correct': 154,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 155,  
&emsp;&emsp;'actual': 156,  
&emsp;&emsp;'precision': 0.9871794871794872,  
&emsp;&emsp;'recall': 0.9935483870967742,  
&emsp;&emsp;'f1': 0.990353697749196},  
&emsp;'strict': {'correct': 131,  
&emsp;&emsp;'incorrect': 23,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 155,  
&emsp;&emsp;'actual': 156,  
&emsp;&emsp;'precision': 0.8397435897435898,  
&emsp;&emsp;'recall': 0.8451612903225807,  
&emsp;&emsp;'f1': 0.842443729903537},  
&emsp;'exact': {'correct': 154,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 2,  
&emsp;&emsp;'possible': 155,  
&emsp;&emsp;'actual': 156,  
&emsp;&emsp;'precision': 0.9871794871794872,  
&emsp;&emsp;'recall': 0.9935483870967742,  
&emsp;&emsp;'f1': 0.990353697749196}},  
 'EVENT_OCCURRENCE': {'ent_type': {'correct': 3370,  
&emsp;&emsp;'incorrect': 746,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 394,  
&emsp;&emsp;'spurious': 960,  
&emsp;&emsp;'possible': 4510,  
&emsp;&emsp;'actual': 5076,  
&emsp;&emsp;'precision': 0.6639085894405043,  
&emsp;&emsp;'recall': 0.7472283813747228,  
&emsp;&emsp;'f1': 0.7031087001877737},  
&emsp;'partial': {'correct': 4116,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 394,  
&emsp;&emsp;'spurious': 960,  
&emsp;&emsp;'possible': 4510,  
&emsp;&emsp;'actual': 5076,  
&emsp;&emsp;'precision': 0.8108747044917257,  
&emsp;&emsp;'recall': 0.9126385809312638,  
&emsp;&emsp;'f1': 0.8587523471729606},  
&emsp;'strict': {'correct': 3370,  
&emsp;&emsp;'incorrect': 746,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 394,  
&emsp;&emsp;'spurious': 960,  
&emsp;&emsp;'possible': 4510,  
&emsp;&emsp;'actual': 5076,  
&emsp;&emsp;'precision': 0.6639085894405043,  
&emsp;&emsp;'recall': 0.7472283813747228,  
&emsp;&emsp;'f1': 0.7031087001877737},  
&emsp;'exact': {'correct': 4116,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 394,  
&emsp;&emsp;'spurious': 960,  
&emsp;&emsp;'possible': 4510,  
&emsp;&emsp;'actual': 5076,  
&emsp;&emsp;'precision': 0.8108747044917257,  
&emsp;&emsp;'recall': 0.9126385809312638,  
&emsp;&emsp;'f1': 0.8587523471729606}},  
 'EVENT_PERCEPTION': {'ent_type': {'correct': 0,  
&emsp;&emsp;'incorrect': 3,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 3,  
&emsp;&emsp;'actual': 3,  
&emsp;&emsp;'precision': 0.0,  
&emsp;&emsp;'recall': 0.0,  
&emsp;&emsp;'f1': 0},  
&emsp;'partial': {'correct': 3,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 3,  
&emsp;&emsp;'actual': 3,  
&emsp;&emsp;'precision': 1.0,  
&emsp;&emsp;'recall': 1.0,  
&emsp;&emsp;'f1': 1.0},  
&emsp;'strict': {'correct': 0,  
&emsp;&emsp;'incorrect': 3,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 3,  
&emsp;&emsp;'actual': 3,  
&emsp;&emsp;'precision': 0.0,  
&emsp;&emsp;'recall': 0.0,  
&emsp;&emsp;'f1': 0},  
&emsp;'exact': {'correct': 3,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 0,  
&emsp;&emsp;'spurious': 0,  
&emsp;&emsp;'possible': 3,  
&emsp;&emsp;'actual': 3,  
&emsp;&emsp;'precision': 1.0,  
&emsp;&emsp;'recall': 1.0,  
&emsp;&emsp;'f1': 1.0}},  
 'EVENT_REPORTING': {'ent_type': {'correct': 225,  
&emsp;&emsp;'incorrect': 31,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 257,  
&emsp;&emsp;'actual': 260,  
&emsp;&emsp;'precision': 0.8653846153846154,  
&emsp;&emsp;'recall': 0.8754863813229572,  
&emsp;&emsp;'f1': 0.8704061895551258},  
&emsp;'partial': {'correct': 256,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 257,  
&emsp;&emsp;'actual': 260,  
&emsp;&emsp;'precision': 0.9846153846153847,  
&emsp;&emsp;'recall': 0.9961089494163424,  
&emsp;&emsp;'f1': 0.9903288201160542},  
&emsp;'strict': {'correct': 225,  
&emsp;&emsp;'incorrect': 31,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 257,  
&emsp;&emsp;'actual': 260,  
&emsp;&emsp;'precision': 0.8653846153846154,  
&emsp;&emsp;'recall': 0.8754863813229572,  
&emsp;&emsp;'f1': 0.8704061895551258},  
&emsp;'exact': {'correct': 256,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 1,  
&emsp;&emsp;'spurious': 4,  
&emsp;&emsp;'possible': 257,  
&emsp;&emsp;'actual': 260,  
&emsp;&emsp;'precision': 0.9846153846153847,  
&emsp;&emsp;'recall': 0.9961089494163424,  
&emsp;&emsp;'f1': 0.9903288201160542}},  
 'EVENT_STATE': {'ent_type': {'correct': 926,  
&emsp;&emsp;'incorrect': 136,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 224,  
&emsp;&emsp;'spurious': 250,  
&emsp;&emsp;'possible': 1286,  
&emsp;&emsp;'actual': 1312,  
&emsp;&emsp;'precision': 0.7057926829268293,  
&emsp;&emsp;'recall': 0.7200622083981337,  
&emsp;&emsp;'f1': 0.7128560431100848},  
&emsp;'partial': {'correct': 1062,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 224,  
&emsp;&emsp;'spurious': 250,  
&emsp;&emsp;'possible': 1286,  
&emsp;&emsp;'actual': 1312,  
&emsp;&emsp;'precision': 0.8094512195121951,  
&emsp;&emsp;'recall': 0.8258164852255054,  
&emsp;&emsp;'f1': 0.817551963048499},  
&emsp;'strict': {'correct': 926,  
&emsp;&emsp;'incorrect': 136,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 224,  
&emsp;&emsp;'spurious': 250,  
&emsp;&emsp;'possible': 1286,  
&emsp;&emsp;'actual': 1312,  
&emsp;&emsp;'precision': 0.7057926829268293,  
&emsp;&emsp;'recall': 0.7200622083981337,  
&emsp;&emsp;'f1': 0.7128560431100848},  
&emsp;'exact': {'correct': 1062,  
&emsp;&emsp;'incorrect': 0,  
&emsp;&emsp;'partial': 0,  
&emsp;&emsp;'missed': 224,  
&emsp;&emsp;'spurious': 250,  
&emsp;&emsp;'possible': 1286,  
&emsp;&emsp;'actual': 1312,  
&emsp;&emsp;'precision': 0.8094512195121951,  
&emsp;&emsp;'recall': 0.8258164852255054,  
&emsp;&emsp;'f1': 0.817551963048499}}}  